# 日级别GARCH测试, iv使用金牛座数据

In [1]:
import numpy as np
import pandas as pd
import dwj_tools.read_hdf as r
from tqdm import tqdm
import warnings

length = 120
warnings.simplefilter('ignore')
a = pd.read_hdf('510050_min.h5')
idx = [240*i+234 for i in range(int(len(a)/240))]
a = a.iloc[idx, :]
b = np.log((a['close']/a['close'].shift(1)).dropna().values)
pred = pd.read_csv(f'b_{length}.csv')
pred.index = a.index[length:]
# und, synf = r.get_und_and_synf_given_range('510050', start_date='20170103', end_date='20221107', csd_range=[234])  # 给定时间范围获取dogsk标的与合成期货数据
# iv_1 = synf['iv_1']
# iv_1.index = a.index[731:]
# pred_start = 
if length==360:
    c = pd.read_excel('gvx2.xlsx', index_col=0).iloc[463+length-1:,:]
else:
    c = pd.read_excel('gvx2.xlsx', index_col=0).iloc[463+length:,:]
diff_iv = pd.read_excel('gvx2.xlsx', index_col=0)['GVX2'].diff().values[463+length-1:]
date = c.index
c.index = [int(a.strftime('%Y%m%d')) for a in c.index]
iv = pred.loc[list(c.index),:]
iv = iv.reset_index()
csd_model = []
csd_pred = []
# 计算相关性使用预测的和实际的iv变化的方向(np.sign)
corr_len = 120
for i in np.arange(corr_len,len(iv)):
    real_diff_iv_hist = np.sign(diff_iv[i-corr_len+1:i+1])
    corr_list = []
    for j in np.arange(2,8):
        model_pred_hist = np.sign(iv.iloc[i-corr_len:i,j].values)
        temp_pred_hist = model_pred_hist[~np.isnan(model_pred_hist)]
        temp_diff = real_diff_iv_hist[~np.isnan(model_pred_hist)]
        corr = np.corrcoef(temp_pred_hist, temp_diff)
        corr_list += [corr[0,1]]
    corr_list = np.array(corr_list)
    temp_pred = np.array(iv.iloc[i,2:])
    corr_list = corr_list[~np.isnan(temp_pred)]
    temp_pred = temp_pred[~np.isnan(temp_pred)]
    csd_model += [corr_list.argmax()]
    csd_pred += [temp_pred[csd_model[-1]]]
count = 0
all_count = 0
for i, temp_pred in enumerate(csd_pred[corr_len:-1]):
    if temp_pred*diff_iv[corr_len+1+i]>=0:
        count += 1
    all_count += 1
print(count/len(csd_pred[corr_len:-1]))

0.5118577075098815


## 作图比较

In [12]:
import plotly.graph_objects as go
import plotly

Fig = go.Figure()
Fig.add_trace(go.Scatter(x=date[120:-1], y=csd_pred[:-1], mode='lines', name='pred'))
Fig.add_trace(go.Scatter(x=date[120:-1], y=diff_iv[121:], mode='lines', name='iv'))

# 主要控制变量在这里
# Fig.update_layout(height=950,
#     			  width=40000,
#     			  xaxis_title=' ',
# 				  xaxis={'tickmode':'auto', 'nticks':1000} # 重点在这里，要自动化tickmode，然后再设置间距.
# )
Fig.show()
plotly.offline.plot(Fig, filename='a.html')

'a.html'

## 根据分位数选择屏蔽条件检验准确性

In [133]:
pred_thre = [50, 60]
abs_pred = np.abs(np.array(csd_pred))
count = 0
all_count = 0
for i, temp_pred in enumerate(abs_pred[corr_len:-1]):
    if np.percentile(abs_pred, pred_thre[0])<=temp_pred<=np.percentile(abs_pred, pred_thre[1]):
        if csd_pred[corr_len+i]*diff_iv[corr_len+1+i]>=0:
            count += 1
        all_count += 1
print(count/all_count)

0.5471698113207547


## 对120, 240, 360循环测试屏蔽的准确性

In [2]:
a = pd.read_hdf('510050_min.h5')
idx = [240*i+234 for i in range(int(len(a)/240))]
a = a.iloc[idx, :]
b = np.log((a['close']/a['close'].shift(1)).dropna().values)
def test(length, pred_thre):
    pred = pd.read_csv(f'b_{length}.csv')
    pred.index = a.index[length:]
    if length==360:
        c = pd.read_excel('gvx2.xlsx', index_col=0).iloc[463+length-1:,:]
    else:
        c = pd.read_excel('gvx2.xlsx', index_col=0).iloc[463+length:,:]
    diff_iv = pd.read_excel('gvx2.xlsx', index_col=0)['GVX2'].diff().values[463+length-1:]
    date = c.index
    c.index = [int(a.strftime('%Y%m%d')) for a in c.index]
    iv = pred.loc[list(c.index),:]
    iv = iv.reset_index()
    csd_model = []
    csd_pred = []
    # 计算相关性使用预测的和实际的iv变化的方向(np.sign)
    corr_len = 120
    for i in np.arange(corr_len,len(iv)):
        real_diff_iv_hist = np.sign(diff_iv[i-corr_len+1:i+1])
        corr_list = []
        for j in np.arange(2,8):
            model_pred_hist = np.sign(iv.iloc[i-corr_len:i,j].values)
            temp_pred_hist = model_pred_hist[~np.isnan(model_pred_hist)]
            temp_diff = real_diff_iv_hist[~np.isnan(model_pred_hist)]
            corr = np.corrcoef(temp_pred_hist, temp_diff)
            corr_list += [corr[0,1]]
        corr_list = np.array(corr_list)
        temp_pred = np.array(iv.iloc[i,2:])
        corr_list = corr_list[~np.isnan(temp_pred)]
        temp_pred = temp_pred[~np.isnan(temp_pred)]
        csd_model += [corr_list.argmax()]
        csd_pred += [temp_pred[csd_model[-1]]]
    abs_pred = np.abs(np.array(csd_pred))
    count = 0
    all_count = 0
    for i, temp_pred in enumerate(abs_pred[corr_len:-1]):
        if np.percentile(abs_pred, pred_thre[0])<=temp_pred<=np.percentile(abs_pred, pred_thre[1]):
            if csd_pred[corr_len+i]*diff_iv[corr_len+1+i]>=0:
                count += 1
            all_count += 1
    try:
        res = count/all_count
    except ZeroDivisionError:
        res = 0
    return res


for csd_len in [120, 240, 360]:
    print(csd_len)
    globals()[f'res_{csd_len}'] = np.zeros((101, 101))
    for i in tqdm(range(101)):
        for j in range(int(i)):
            globals()[f'res_{csd_len}'][j, i] = test(csd_len, [j, i])

  0%|          | 0/101 [00:00<?, ?it/s]

120


  0%|          | 0/101 [00:00<?, ?it/s]

240


  0%|          | 0/101 [00:00<?, ?it/s]

360


100%|██████████| 101/101 [1:24:11<00:00, 50.01s/it]


### 找出上述不同周期中准确率最高的thre

In [10]:
thre_tol = 25
for csd_horizon in [120, 240, 360]:
    for csd_i in range(75):
        temp_df = globals()[f'res_{csd_horizon}'][csd_i,csd_i+thre_tol:]
        print(temp_df.max())

0.5212355212355212
0.5121457489878543
0.5138178096212896
0.5113871635610766
0.5094339622641509
0.5090137857900318
0.5085836909871244
0.5108695652173914
0.5137513751375138
0.5133630289532294
0.5124153498871332
0.512
0.5109826589595375
0.5105386416861827
0.5088757396449705
0.5119617224880383
0.5157766990291263
0.5135135135135135
0.513681592039801
0.5164141414141414
0.5146871008939975
0.5097024579560155
0.5111111111111111
0.512549537648613
0.5106666666666667
0.5114709851551957
0.5122950819672131
0.5159500693481276
0.5183098591549296
0.51931330472103
0.5159883720930233
0.5206489675516224
0.5179640718562875
0.5212765957446809
0.5215384615384615
0.5195618153364632
0.5238853503184714
0.5274193548387097
0.5287356321839081
0.5301003344481605
0.5246179966044142
0.5251299826689775
0.5247349823321554
0.5225225225225225
0.5267034990791897
0.5300751879699248
0.5335892514395394
0.5343811394891945
0.5381526104417671
0.5359342915811088
0.5387840670859538
0.5407725321888412
0.5384615384615384
0.53153153

## 效果不佳, 换用前一段时间的预测准确率来选择所用模型

In [18]:
# def test_2(garch_length, corr_len, pred_thre):
garch_length = 240
corr_len = 120
pred_thre = [0.49, 0.5]
length = garch_length
a = pd.read_hdf('510050_min.h5')
idx = [240*i+234 for i in range(int(len(a)/240))]
a = a.iloc[idx, :]
b = np.log((a['close']/a['close'].shift(1)).dropna().values)
pred = pd.read_csv(f'b_{length}.csv')
pred.index = a.index[length:]
if length==360:
    c = pd.read_excel('gvx2.xlsx', index_col=0).iloc[463+length-1:,:]
    diff_iv = pd.read_excel('gvx2.xlsx', index_col=0)['GVX2'].diff().values[463+length-1:]
else:
    c = pd.read_excel('gvx2.xlsx', index_col=0).iloc[463+length:,:]
    diff_iv = pd.read_excel('gvx2.xlsx', index_col=0)['GVX2'].diff().values[463+length:]
date = c.index
c.index = [int(a.strftime('%Y%m%d')) for a in c.index]
iv = pred.loc[list(c.index),:]
iv = iv.reset_index()
csd_model = []
csd_pred = []
# 计算相关性使用预测的和实际的iv变化的方向(np.sign)
for i in np.arange(corr_len,len(iv)):
    real_diff_iv_hist = np.sign(diff_iv[i-corr_len+1:i+1])
    corr_list = []
    for j in np.arange(2,8):
        model_pred_hist = np.sign(iv.iloc[i-corr_len:i,j].values)
        temp_pred_hist = model_pred_hist[(~np.isnan(model_pred_hist))]
        temp_diff = real_diff_iv_hist[(~np.isnan(model_pred_hist))]
        temp_diff = temp_diff[(temp_pred_hist<pred_thre[1])*(temp_pred_hist>pred_thre[0])]
        temp_pred_hist = temp_pred_hist[(temp_pred_hist<pred_thre[1])*(temp_pred_hist>pred_thre[0])]
        corr_rate = (temp_pred_hist*temp_diff>=0).sum()/len(temp_pred_hist)
        corr_list += [corr_rate]
    corr_list = np.array(corr_list)
    temp_pred = np.array(iv.iloc[i,2:])
    corr_list = corr_list[~np.isnan(temp_pred)]
    temp_pred = temp_pred[~np.isnan(temp_pred)]
    csd_model += [corr_list.argmax()]
    csd_pred += [temp_pred[csd_model[-1]]]
count = 0
all_count = 0
for i, temp_pred in enumerate(csd_pred[corr_len:-1]):
    if temp_pred*diff_iv[corr_len+1+i]>=0:
        count += 1
    all_count += 1
print(count/len(csd_pred[corr_len:-1]))

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


## 使用16个模型的预测值做上述统计

In [ ]:
import dwj_tools 

In [16]:
# def test_2(garch_length, corr_len, pred_thre):
garch_length = 240
corr_len = 360
pred_thre = [0.02, 0.5]
length = garch_length
a = pd.read_hdf('510050_min.h5')
idx = [240*i+234 for i in range(int(len(a)/240))]
a = a.iloc[idx, :]
b = np.log((a['close']/a['close'].shift(1)).dropna().values)
pred = pd.read_csv(f'b_all_{length}.csv')
pred.index = a.index[length:]
if length==360:
    c = pd.read_excel('gvx2.xlsx', index_col=0).iloc[463+length-1:,:]
    diff_iv = pd.read_excel('gvx2.xlsx', index_col=0)['GVX2'].diff().values[463+length-1:]
else:
    c = pd.read_excel('gvx2.xlsx', index_col=0).iloc[463+length:,:]
    diff_iv = pd.read_excel('gvx2.xlsx', index_col=0)['GVX2'].diff().values[463+length:]
date = c.index
c.index = [int(a.strftime('%Y%m%d')) for a in c.index]
iv = pred.loc[list(c.index),:]
iv = iv.reset_index()
csd_model = []
csd_pred = []
# 计算相关性使用预测的和实际的iv变化的方向(np.sign)
for i in np.arange(corr_len,len(iv)):
    real_diff_iv_hist = np.sign(diff_iv[i-corr_len+1:i+1])
    corr_list = []
    for j in np.arange(1,16):
        model_pred_hist = np.sign(iv.iloc[i-corr_len:i,j].values)
        temp_pred_hist = model_pred_hist[(~np.isnan(model_pred_hist))]
        temp_diff = real_diff_iv_hist[(~np.isnan(model_pred_hist))]
        temp_diff = temp_diff[(temp_pred_hist<pred_thre[1])*(temp_pred_hist>pred_thre[0])]
        temp_pred_hist = temp_pred_hist[(temp_pred_hist<pred_thre[1])*(temp_pred_hist>pred_thre[0])]
        corr_rate = (temp_pred_hist*temp_diff>=0).sum()/len(temp_pred_hist)
        corr_list += [corr_rate]
    corr_list = np.array(corr_list)
    temp_pred = np.array(iv.iloc[i,1:])
    corr_list = corr_list[~np.isnan(temp_pred)]
    temp_pred = temp_pred[~np.isnan(temp_pred)]
    csd_model += [corr_list.argmax()]
    csd_pred += [temp_pred[csd_model[-1]]]
count = 0
all_count = 0
for i, temp_pred in enumerate(csd_pred[corr_len:-1]):
    if temp_pred*diff_iv[corr_len+1+i]>0:
        count += 1
    all_count += 1
print(count/len(csd_pred[corr_len:-1]))

0.5048543689320388
